In [1]:
import pandas as pd
import numpy as np

# 可視化用のライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import neologdn
import MeCab

import re

In [2]:
df = pd.read_excel('sample_text.xlsx')
df.head()

,内容
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
1,原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される
2,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、...
3,次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid...


In [3]:
text_df = df[['内容']]
text_df.head()

,内容
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
1,原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される
2,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、...
3,次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid...


## 形態素解析結果をarray形式で出力する関数を定義
後にarrayをデータフレーム形式に変換するための準備

形態素解析をする関数を全文章に適応させる際に返り値がarrayになる関数を使った時の方が処理スピードが速いため

In [4]:
def get_wakati_text_array(text):
    # 【neologdnで文字表現の正規】
    text_normalization = neologdn.normalize(text)
    
    # 【【形態素解析】】
    # 【辞書設定】
    #　開発環境がwindowsの場合：'-Ochasen -d C:\mecab-ipadic-neologd'
    # 開発環境がMacの場合:'-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd'
    neologd_tagger = MeCab.Tagger('-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
    # 【形態素解析を実行、それをスペースあり改行なしの文字列にする】
    # neologd_tagger.parse(text)で下記のように各単語の原形、品詞などが1行に連続して表示される
    # [表示\tヒョウジ\t表示\t名詞-サ変接続\t\t\nさ\t　.....\nEOS\n']
    # 原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される
    #　neologd_tagger.parseで形態素解析した後、形態素解析結果を「'\n'→'\t'」に変換することで単にスペースがある改行なしの１つの文字列になる
    # この文字列に対して「'\t'」でsplitすることで「単語」「その読み方」「その品詞」... が1つ1つのの要素になったリスト型になる
    # このリストの最後の２つの要素は「EOS」と「(空白文字)」であり、形態素解析結果として使用しないためこれら２つを含まないように抽出する→[:-2]
    wakati_1d_text = neologd_tagger.parse(text_normalization).replace('\n', '\t').split('\t')[:-2]
    
    # 【【形態素解析結果をデータフレームに格納する準備】】
    # この時点でwakati_1d_textは1次元のリストであり、このままの形式でpd.DataFrame関数の引数に渡してしまうと１列のデータフレームになってしまう
    # そこでnumpy.array関数とreshape関数を使い下記のような[n行（単語数分）６列(単語、読み方、原形、品詞、活用形、活用型)」のn次元配列リストを作成
    # ['原形', 'ゲンケイ', '原形', '名詞-一般', '', ''],
    # ['、', '、', '、', '記号-読点', '', ''],
    # この[n行（単語数分）６列(単語、読み方、原形、品詞、活用形、活用型)」のn次元配列リストをpd.DataFrame関数の引数に渡す
    wakati_array = np.array([wakati_1d_text]).reshape(-1, 6)
    return wakati_array

## 全文章に対し形態素解析結果をarray形式で出力する関数を実行

In [5]:
# [内容]カラムの全行に対してget_wakati_text_array関数を実行
# 行毎に形態素解析結果が['原形', 'ゲンケイ', '原形', '名詞-一般', '', '']　といった形式で格納される
wakati_text_array = text_df['内容'].apply(get_wakati_text_array)

In [6]:
# 確認
wakati_text_array

0    [[neologd, ネオログディー, NEologd, 名詞-固有名詞-一般, , ], ...
1    [[原形, ゲンケイ, 原形, 名詞-一般, , ], [、, 、, 、, 記号-読点, ,...
2    [[まずは, マズハ, まずは, 接続詞, , ], [parse, パース, Parse,...
3    [[次に, ツギニ, 次に, 接続詞, , ], [邪魔, ジャマ, 邪魔, 名詞-形容動詞...
Name: 内容, dtype: object

## arrayからDataFrameに変換

In [7]:
# wakati_text_arrayから１要素（リストになっている）毎取り出し、それをデータフレーム型にするリスト内包表記
# wakati_df_listには形態素解析結果を格納したデータフレームが１要素として格納される
wakati_df_list = [pd.DataFrame(wakati_text) for wakati_text in wakati_text_array]

In [8]:
# 確認
wakati_df_list

[          0        1        2           3      4       5
 0   neologd  ネオログディー  NEologd  名詞-固有名詞-一般               
 1         _        _        _       記号-一般               
 2    tagger      タガー   tagger  名詞-固有名詞-一般               
 3         .        .        .       記号-一般               
 4     parse      パース    Parse  名詞-固有名詞-一般               
 5         (        (        (       記号-一般               
 6      text     テクスト     text  名詞-固有名詞-一般               
 7         )        )        )       記号-一般               
 8         で        デ        で   助詞-格助詞-一般               
 9         各       カク        各    接頭詞-名詞接続               
 10       単語      タンゴ       単語       名詞-一般               
 11        の        ノ        の      助詞-連体化               
 12       原形     ゲンケイ       原形       名詞-一般               
 13        、        、        、       記号-読点               
 14       品詞      ヒンシ       品詞       名詞-一般               
 15       など       ナド       など      助詞-副助詞               
 16        が  

## 形態素解析後の各単語(や原形、品詞等)と形態素解析前の文章の紐付け

In [9]:
# wakati_df_listには形態素解析結果を格納したデータフレームが１要素として格納されている
# しかしながら、このままだとどの単語がどの文章から形態素解析された単語であったかどうか確認することができない
# そこで、形態素解析された単語に対し元の文章を紐付けていく
# text_df['内容']:Series型　　Series型に「.values」を追加するとarrayに変換できる
# 今回の場合はtext_df['内容']列１つに対してarray変換したので１次元配列リスト
# そのリストの要素は文章１つ１つでインデックス0番目に最初の文章が格納されている
# よって、このtext_df['内容'].valuesのn番目の文章とwakati_df_listのn番目のデータフレームは紐づいていることになる
#　以上より、複数のリストの要素を同時に取得できるzip関数を使用しこれら2つを同時に取り出し紐付けるfor文を作成
#　その際、assign関数でデータフレーム（変数b）に新しいカラム[内容]を追加、そのカラムに文章（変数a）を割り当てる
array_df_tmp = []
for a, b in zip(text_df['内容'].values, wakati_df_list):
    array_df_tmp.append(b.assign(内容 = a))

In [10]:
# 確認(確認しやすいように０番目の要素のみ）
array_df_tmp[0]

,0,1,2,3,4,5,内容
0,neologd,ネオログディー,NEologd,名詞-固有名詞-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
1,_,_,_,記号-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
2,tagger,タガー,tagger,名詞-固有名詞-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
3,.,.,.,記号-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
4,parse,パース,Parse,名詞-固有名詞-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
5,(,(,(,記号-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
6,text,テクスト,text,名詞-固有名詞-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
7,),),),記号-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
8,で,デ,で,助詞-格助詞-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
9,各,カク,各,接頭詞-名詞接続,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...


## リストに格納されたデータフレームをすべて連結

In [11]:
# larray_df_tmpリストの中に形態素解析後の結果を格納したデータフレームが全文章分ある
# よって、このリストをpd.concat関数の引数に与えることでそれらをまとめて１度で連結できる
# axis=0 : 縦に連結
all_wakati_df = pd.concat(array_df_tmp, axis=0)

In [12]:
# concatされたか確認
all_wakati_df.head(50)

,0,1,2,3,4,5,内容
0,neologd,ネオログディー,NEologd,名詞-固有名詞-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
1,_,_,_,記号-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
2,tagger,タガー,tagger,名詞-固有名詞-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
3,.,.,.,記号-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
4,parse,パース,Parse,名詞-固有名詞-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
5,(,(,(,記号-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
6,text,テクスト,text,名詞-固有名詞-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
7,),),),記号-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
8,で,デ,で,助詞-格助詞-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
9,各,カク,各,接頭詞-名詞接続,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...


## データフレームのカラム名を変更しcsvファイルで出力

In [13]:
# カラム名を変更
all_wakati_df = all_wakati_df.rename(columns = {0:'surface', 1:'yomi', 2:'original', 3:'type', 4:'katsuyoukei', 5:'katsuyougata'})
all_wakati_df.head()

,surface,yomi,original,type,katsuyoukei,katsuyougata,内容
0,neologd,ネオログディー,NEologd,名詞-固有名詞-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
1,_,_,_,記号-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
2,tagger,タガー,tagger,名詞-固有名詞-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
3,.,.,.,記号-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
4,parse,パース,Parse,名詞-固有名詞-一般,,,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...


In [14]:
# csvファイルとして出力
all_wakati_df.to_csv('all_text_wakatigaki.csv', index=False, encoding='utf-8')